In [15]:
import os
import sys
from astropy.table import Table
from desisurvey.progress import Progress
import desimodel.io


In [16]:
os.environ['DESISURVEY_OUTPUT'] = os.path.join('/global/cscratch1/sd/forero/quicksurvey2017b/survey')

In [23]:
desisurvey_path = os.getenv('DESISURVEY_OUTPUT')

if desisurvey_path is None:
    print('Environment variable DESISURVEY_OUTPUT is not set')
else:
    print('Environment variable DESISURVEY_OUTPUT set to {}'.format(desisurvey_path))
    Progress(restore='progress.fits').get_exposures().write(os.path.join(desisurvey_path,'exposures.fits'), overwrite=True)
    
    explist = Table.read(os.path.join(desisurvey_path,'exposures.fits'))
    # separate the exposures for the different programs
    isbright = explist['pass'] > 4 
    isgray = explist['pass'] == 4
    isdark = explist['pass'] < 4
    
    tiles = desimodel.io.load_tiles()


Environment variable DESISURVEY_OUTPUT set to /global/cscratch1/sd/forero/quicksurvey2017b/survey
INFO:progress.py:145:__init__: Loaded progress from /global/cscratch1/sd/forero/quicksurvey2017b/survey/progress.fits.


NameError: name 'desimodel' is not defined

In [18]:
!ls

MyQuota  desi  exposures.fits  quicksurvey2017b  quicksurvey_example


In [ ]:
#get the dates when fiberassign should be run

px = Table.read(os.getenv('DESISURVEY_OUTPUT')+'/progress.fits')
fiberassign_dates = np.sort(list(set(px['available'])))
fiberassign_dates = fiberassign_dates[fiberassign_dates>0]
print(fiberassign_dates)

#load the dark exposures
exposurefile = os.path.join(surveydir,'exposures_dark.fits')
exposures = Table.read(exposurefile)
expdates = []
for n in exposures['night']:
    a = datetime.datetime.strptime(n, "%Y-%m-%d")
    expdates.append(a.date())
expdates = np.array(expdates)
print(expdates, expdates)

#load the first date of the survey
survey_config = os.path.join(os.getenv('DESISURVEY_OUTPUT'),'config.yaml')
with open(survey_config, 'r') as pfile:
    params = yaml.load(pfile)
pfile.close()
first_day = params['first_day']
last_day = params['last_day']

#compute the dates to run fiberassign to be sure that there is at least one exposure there
fiberassign_dates_dark = []
one_day = datetime.timedelta(days=1)
initial_day = first_day #expdates.min()
for d in fiberassign_dates:
    final_day = first_day + d * one_day
    ii = (expdates > initial_day) & (expdates<=final_day)
    n_in = np.count_nonzero(ii)
    if n_in>0:
        print(initial_day, final_day, n_in)
        fiberassign_dates_dark.append(initial_day.strftime("%Y-%m-%d"))
        initial_day = final_day

print(fiberassign_dates_dark)
fiberdates_filename = os.path.join(fiberassigndir, 'fiberassign_dates_dark.txt')
f = open(fiberdates_filename, 'w')
for d in fiberassign_dates_dark:
    f.write(d+"\n")
f.close()